<a href="https://colab.research.google.com/github/LeoDardanello/DA_Semantic_Segmentation/blob/main/Semantic_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys, os
import numpy as np
import torchvision

drive.mount('/content/drive')


In [ ]:
!7z x '/content/drive/MyDrive/AML/Cityscapes.zip'

In [ ]:
if os.path.exists("/content/DA_Semantic_Segmentation"):
  !rm -rf "/content/DA_Semantic_Segmentation"
!git clone "https://github.com/LeoDardanello/DA_Semantic_Segmentation.git"

**TRAINING**

In [ ]:
!pip install tensorboardX

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

# GTA 5 TEST


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import ToPILImage


transform=ToPILImage()
image,label=GATTO[0]
image=transform(image)


# if not torch.all(label>=0 & label<19 | label==255):
#   print("ERRORE")

plt.imshow(np.array(image))
plt.show()
plt.imshow(label.numpy(),cmap="gray")
plt.show()

In [ ]:
!7z x '/content/drive/MyDrive/AML/GTA5.zip'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --dataset_train "GTA5" --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

In [10]:
!rm -rf "/content/GTA5/TrainID"

# GTA TRAINING WITH CITYSCAPE TESTING

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --batch_size 8 --learning_rate 0.001 --num_workers 2 --dataset_train "GTA5" --dataset_test "Cityscapes" --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_7391.tar"

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train_with_DA.py" --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"